In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

from netgen.occ import *
from netgen.csg import *

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp


In [2]:
#do the netgen meshing in a cell before importing ngsolve 
help(unit_cube)
netgen_mesh = unit_cube.GenerateMesh(maxh=1)
#netgen_mesh.GenerateVolumeMesh() #for uniform meshes this changes nothing?

#netgen_mesh.Refine()

mesh = Mesh(netgen_mesh)

# mesh.Refine()
# mesh.Refine()
# mesh.Refine()
#for meshsize = 0.125 need to redo the matrix multiplication for B
#mesh.Refine()
#NGsolve mesh

# print(mesh.nedge)
# for v in mesh.vertices:
#     print (v, v.point)

# for el in mesh.Elements(VOL):
#     print(type(el))
#     print ("vertices: ", el.vertices)   # get vertices of an element
#     print ("edges: ", el.edges)         # get edges of an element
#help(mesh)
print(mesh.nface)
print(mesh.ne)
Draw(mesh)




Help on CSGeometry in module netgen.libngpy._csg object:

class CSGeometry(netgen.libngpy._meshing.NetgenGeometry)
 |  Method resolution order:
 |      CSGeometry
 |      netgen.libngpy._meshing.NetgenGeometry
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Add(...)
 |      Add(self: netgen.libngpy._csg.CSGeometry, solid: netgen.libngpy._csg.Solid, bcmod: list = [], maxh: float = 1e+99, col: tuple = (), transparent: bool = False, layer: int = 1) -> int
 |  
 |  AddPoint(...)
 |      AddPoint(self: netgen.libngpy._csg.CSGeometry, arg0: netgen.libngpy._meshing.Point3d, arg1: Union[int, str]) -> netgen.libngpy._csg.CSGeometry
 |  
 |  AddSplineSurface(...)
 |      AddSplineSurface(self: netgen.libngpy._csg.CSGeometry, SplineSurface: netgen.libngpy._csg.SplineSurface) -> None
 |  
 |  AddSurface(...)
 |      AddSurface(self: netgen.libngpy._csg.CSGeometry, surface: netgen.libngpy._csg.Solid, solid: netgen.libngpy._csg.Solid) -> None

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

Set correct functionspaces in NGSolve, which will give the corresponding FE Elements, 

In [5]:
#set function space,
#help(HCurl(mesh, order = 1, type1=True, complex=True))
#for complex= false we can visalize the dofs 
H_h = Compress(HCurl(mesh, order = 1, type1=True, complex=False)) # main function space
H_H = Compress(HCurl(mesh, order = 2, type1=True, complex=False)) # high order Function space for Riesz representative
#print(H_h.globalorder)

H = L2(mesh, order = 0) 



# gfu = GridFunction(H_h,multidim = H_h.ndof)
# help(gfu)
# for i in range(H_h.ndof):
#     gfu.vecs[i][i] = 1.0
# Draw(gfu, animate = True, min = 0, max = 2.0, deformation = False, euler_angles =[-45,-6,25])

print(H.ndof)



12


In [ ]:
#choose w_h and set its values on the nodes
w = (exp(2j * z),0,0)

w_h = GridFunction(H_h) #set the nodes
w_h.Set(w)

f_H = GridFunction(H_H, name="f_H") #high order 

Draw(w_h)

Create the galerkin matrices used to calculate the Riesz representative

In [ ]:
#low order
u_h = H_h.TrialFunction() # define the trial function
v_h = H_h.TestFunction() # define the test function

#High order
u_H = H_H.TrialFunction() # define the trial function
v_H= H_H.TestFunction() # define the test function

m = BilinearForm(H_H) #define m with the L2 inner product on the high order space
m += u_H*v_H * dx 
m.Assemble()

a = BilinearForm(H_H) # define the H(curl) inner product on the high order space
a += curl(u_H) * curl(v_H) * dx
a += u_H*v_H * dx 
a.Assemble()

Approximate the dual norm, by the Riesz representive.
Get the Riesz Representive by solving a LGS, since we're approximating we set the space of the Riesz represntative in a higher order space (higher polynomial degree)

In [ ]:
#create embedding matrix T

# a_mixed = BilinearForm(trialspace=H_h, testspace=H_H)

# a_dest = BilinearForm(H_H)

# v_H_dual = v_H.Operator("dual")

# dS = dx(element_boundary=True)
# a_dest += u_H*v_H_dual*dx + u_H*v_H_dual*dS
# a_dest.Assemble()

# a_mixed += u_h*v_H_dual*dx + u_h*v_H_dual*dS
# a_mixed.Assemble()

# #build T
# t = a_dest.mat.Inverse() @ a_mixed.mat


############################# doing it directly as a L2 projection

a_mixed = BilinearForm(trialspace=H_h, testspace=H_H)
a_mixed += u_h*v_H*dx
a_mixed.Assemble()

a_dest = BilinearForm(H_H)
a_dest += u_H*v_H*dx
a_dest.Assemble()

t = a_dest.mat.Inverse() @ a_mixed.mat

#w_H is the high order embedding of w_h
w_H = GridFunction(H_H, name="w_H")
w_H.vec.data = t * w_h.vec

Draw(w_h)
Draw(w_H)

######################## this works too ?

#this feels weird, find way to get the range from space H_h itself
# mixed = FESpace([H_h])

# emb = Embedding(H_H.ndof, mixed.Range(0))

# #w_emb = GridFunction(H_H, name="w_emb")
# #w_emb.vec.data = emb * w_h.vec 

# w_H.vec.data = emb * w_h.vec

# Draw(w_H)

#results in same shape
#print(t.shape)
#print(emb.shape)


In [ ]:
a_inv = a.mat.Inverse(freedofs=H_H.FreeDofs()) #, inverse="sparsecholesky")
#does the same as below
#f_H.vec.data = a_inv @ m.mat * w_H.vec
#Draw(f_H)

#solve for the riesz representative (f_H) 
temp = f_H.vec.CreateVector()
temp.data = m.mat * w_H.vec
f_H.vec.data = a_inv * temp
Draw(f_H)



then plug the Riesz representive in the GEV problem and solve it, where the largest Eigenvalue will be the constant we're looking for

In [ ]:
#create Matrices for GEV problem

#c is the low order galerkin matrix
c = BilinearForm(H_h)
c += curl(u_h) * curl(v_h) * dx
c += u_h*v_h * dx 
c.Assemble()

#check norms
# temp3 = c.mat * w_h.vec
# w_h_norm = w_h.vec.InnerProduct(temp3)
# print("H(curl) norm of w_h:", sqrt(w_h_norm)) #norm of w_h in H(div,

# temp4 = a.mat * f_H.vec
# f_H_norm = f_H.vec.InnerProduct(temp4)
# print("H(curl) norm of f_H, the riesz representative of w_h:", sqrt(f_H_norm)) #norm of the riesz representative

#this results in a slightly diffrent Eigenvalue (very small difference)
#b for old(wrong) exp
b = t.T @ m.mat.T @ a_inv @ m.mat @ t

B_dir = b.ToDense().NumPy()

# print(B)

C = c.mat.ToDense().NumPy()
M = m.mat.ToDense().NumPy()
#A = a.mat.ToDense().NumPy()
A_inv = a_inv.ToDense().NumPy()
# print(sp.linalg.inv(A))
# print("\n")
# print(A_inv)
T = t.ToDense().NumPy()

#M == M.T 
#for fine meshes, this is too slow
#b for old(wrong) exp
B = T.T @ M @ A_inv @ M @ T

# print(B)

In [ ]:
#solve GEV using the scipy solver s


y = sp.linalg.eigvalsh(B,C)
print("maximal Eigenvalue:",max(y))
print("minimal Eigenvalue:",min(y))

y = sp.linalg.eigvalsh(B_dir,C)
print("maximal Eigenvalue:",max(y))
print("minimal Eigenvalue:",min(y))